In [1]:
import pandas as pd
from IPython.core.display import display, HTML

all_bids_ids = pd.read_csv("nfb_subset_SUBJECT_IDS.csv")
all_bids_ids['ID'] = all_bids_ids['ID'].str[4:]
all_bids_ids['conn_ID'] = all_bids_ids.index + 1

all_score_ids = pd.read_csv("events_IDs.csv")

bids_subset_ids = pd.merge(all_score_ids, all_bids_ids, how='inner', on='ID').sort_values(by=['ID']).reset_index(drop=True)
display(bids_subset_ids)

,ID,conn_ID
0,A00028185,1
1,A00032875,3
2,A00033747,4
3,A00034854,5
4,A00035072,6
...,...,...
133,A00066827,151
134,A00066926,152
135,A00072203,153
136,A00073600,154


In [15]:
"""

.. gc-fmri

================================
Granger 'causality' of fMRI data
================================

Granger 'causality' analysis provides an asymmetric measure of the coupling
between two time-series. When discussing this analysis method, we will put the
word 'causality' in single quotes, as we believe that use of this word outside
of quotes should be reserved for particular circumstances, often not fulfilled
in the analysis of simultaneously recorder neuroscientific time-series (see
[Pearl2009]_ for an extensive discussion of this distinction).

The central idea behind this analysis is that time-series can be described in
terms of a time-delayed auto-regressive model of the form:

.. math::

   x_t = \sum_{i=1}^{n}a_i x_{t-i} + \epsilon_t

Here, the past behaviour of a single time-series is used in order to predict
the current value of the time-series. In Granger 'causality' analysis, we test
whether the addition of a prediction of the time-series from another
time-series through a multivariate auto-regressive model may improve our
prediction of the present behavior of the time-series (reducing the value of
the error term $\epsilon_t$):

.. math::

   x_t = \sum_{i=1}^{n}(a_i x_{t-i} + b_i y_{t-i}) + \epsilon_t


In our implementation of the algorithms used for this analysis, we follow
closely the description put forth by Ding et al. ([Ding2006]_). Also, see
:ref:`mar` and :ref:`ar` for examples even more closely modeled on the
examples mentioned in their paper.

Here, we will demonstrate the use of Granger 'causality' analysis with fMRI
data. The data is provided as part of the distribution and is taken from a
'resting state' scan. The data was motion corrected and averaged from several
ROIs.

We start by importing the needed modules:

"""

import os

import numpy as np
import matplotlib.pyplot as plt

import nitime
import nitime.analysis as nta
import nitime.timeseries as ts
import nitime.utils as tsu
from nitime.viz import drawmatrix_channels

from IPython.core.display import display, HTML

"""
We then define a few parameters of the data: the TR and the bounds on the
frequency band of interest.
"""

TR = 2.00
f_ub = 0.15
f_lb = 0.02

path = './timeseries_CSVs'
files = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if 'ROI_Subject' in file:
            files.append(os.path.join(r, file))
X = np.array([np.zeros(435)]) #quite possibly unnecessarily complicated, unsure how to do this
for fnum in range(len(files)):
    if int(files[fnum][29:32]) in bids_subset_ids.conn_ID.values:
#         print(files[fnum][29:32])
        data_rec = np.genfromtxt(files[fnum], dtype=float, delimiter=',', names=True)

        roi_names = np.array(data_rec.dtype.names)
        nseq = len(roi_names)
        n_samples = data_rec.shape[0]
        data = np.zeros((nseq, n_samples))

        for n_idx, roi in enumerate(roi_names):
            data[n_idx] = data_rec[roi]
    #     display(data.shape)
    #     if np.isnan(data).any() or np.isinf(data).any():
    #         display(fnum)
        """
        We normalize the data in each of the ROIs to be in units of % change and
        initialize the TimeSeries object:
        """
        pdata = tsu.percent_change(data)
        time_series = ts.TimeSeries(pdata, sampling_interval=TR)
        """
        We initialize the GrangerAnalyzer object, while specifying the order of the
        autoregressive model to be 1 (predict the current behavior of the time-series
        based on one time-point back).
        """
        G = nta.GrangerAnalyzer(time_series, order=1)
        """
        We are only interested in the physiologically relevant frequency band
        (approximately 0.02 to 0.15 Hz).

        The spectral resolution is different in these two different analyzers. In the
        CoherenceAnalyzer, the spectral resolution depends on the size of the window
        used for calculating the spectral density and cross-spectrum, whereas in the
        GrangerAnalyzer it is derived, as determined by the user, from the MAR model
        used.

        For this reason, the indices used to access the relevant part of the spectrum
        will be different in the different analyzers.
        """
        freq_idx_G = np.where((G.frequencies > f_lb) * (G.frequencies < f_ub))[0]
        try:
            y_minus_x = np.mean(G.causality_xy[:, :, freq_idx_G] - G.causality_yx[:, :, freq_idx_G], -1)
            y_minus_x = y_minus_x[np.logical_not(np.isnan(y_minus_x))]

            temp = np.array([y_minus_x.tolist()])
            X = np.concatenate((X, temp), axis=0)
        except:
            print("an error occured that prevented causality calculation for", files[fnum])
    #     display(files[fnum])
X = X[1:]
display(X.shape)
# need to put all causality values in a single array and save it to npy file
# probably need to make an empty array of the correct size first, as in read_matrices.ipynb

an error occured that prevented causality calculation for ./timeseries_CSVs/ROI_Subject080_Session001.csv
an error occured that prevented causality calculation for ./timeseries_CSVs/ROI_Subject102_Session001.csv
an error occured that prevented causality calculation for ./timeseries_CSVs/ROI_Subject111_Session001.csv


(135, 435)